## Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split


from IPython.display import Audio
# import resampy

## Load dataset.csv

In [2]:
def convertPath(current_directory,path):
    """
    Converts to absolute path
    """
    combined_path = os.path.join(current_directory,path)
    absolute_path = os.path.abspath(combined_path)
    return absolute_path

In [3]:
current_directory = os.getcwd()
path_to_dataset_csv = "csv/dataset.csv"

df = pd.read_csv(path_to_dataset_csv)
# Convert to absolute path
df['Path'] = df['Path'].apply(lambda x: convertPath(current_directory,x))

path = np.array(df.Path)[1]
data, sample_rate = librosa.load(path)

# Data Augmentation

In [ ]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(y=data, rate=rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=pitch_factor)

In [ ]:
x = pitch(data, sample_rate)
plt.figure(figsize=(14,4))
librosa.display.waveshow(y=x, sr=sample_rate)
Audio(x, rate=sample_rate)

# Feature Extraction

In [ ]:

def extract_features(data, sample_rate):
    result = np.array([])

    # ZCR
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result = np.hstack((result, zcr))

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft))

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc))

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms))

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel))
    
    # Spectral Centroid
    spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, spectral_centroids))

    # Spectral Bandwidth
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, spectral_bandwidth))

    # Spectral Contrast
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, spectral_contrast))

    # Spectral Rolloff
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, spectral_rolloff))

    # Chroma Deviation
    chroma_std = np.std(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_std))

    return result

def get_features(path):
    # TODO - Consider removing quiet parts of the audio before stitching the audio by 0.5 seconds lengths
    # This will require some calculations to get the duration and stepsize right
    
    hstack = None
    step = 5
    for i in range(0,30,step):
        
        if i == 0:
            offset=0
        else:
            i/=10
            offset = i-step/10
        print(i,offset)

        # Split audio according to the stepsize and offset
        data, sample_rate = librosa.load(path, offset=offset, duration=step/10)
        print(data)
    
        # without augmentation
        res1 = extract_features(data,sample_rate)
        result = np.array(res1)
        
        # data with noise
        noise_data = noise(data)
        res2 = extract_features(noise_data,sample_rate)
        result = np.vstack((result, res2)) # stacking vertically
        
        # data with stretching and pitching
        new_data = stretch(data)
        data_stretch_pitch = pitch(new_data, sample_rate)
        res3 = extract_features(data_stretch_pitch,sample_rate)
        result = np.vstack((result, res3)) # stacking vertically
        
        if i == 0:
            hstack = result
        else: hstack = np.hstack((hstack,result))
        print(hstack.shape)
        
    return hstack

In [ ]:
X, Y = [], []
for path, emotion in zip(df.Path, df.Emotions):
    feature = get_features(path)
    for ele in feature:
        X.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)

In [ ]:
len(X), len(Y), df.Path.shape

# Feature selection

# Save features to csv

In [90]:
Features = pd.DataFrame(X)
Features['labels'] = Y

# Use this naming convension
# author_number-of-features_done-PCA?.csv
# e.g. sam_10_1.csv > Author: sam, 10 features, has done PCA
Features.to_csv('csv/your_filename.csv', index=False)
